# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time 

# Import API key
from config import api_key #congig is the name of the file and api_key is the name of the variable within the file

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("../output_data/cities_data.csv")  #this is the csv I created in WeatherPy that holds results of the cities found
weather_data #test to display the dataframe

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Ushuaia,-54.8000,-68.3000,51.80,43,75,23.02,AR,1611604109
1,Yar-Sale,66.8333,70.8333,-20.42,87,100,15.05,RU,1611604110
2,Punta Arenas,-53.1500,-70.9167,51.80,46,75,24.16,CL,1611604110
3,Hithadhoo,-0.6000,73.0833,81.63,73,67,3.74,MV,1611604110
4,Binzhou,37.3667,118.0167,31.59,93,90,2.86,CN,1611604111
...,...,...,...,...,...,...,...,...,...
574,Luga,58.7372,29.8453,33.94,96,100,5.50,RU,1611604226
575,Norman Wells,65.2820,-126.8329,-16.60,83,75,4.61,CA,1611604226
576,Batagay,67.6333,134.6333,-43.98,75,86,1.34,RU,1611604226
577,Chapecó,-27.0964,-52.6183,71.60,100,75,4.61,BR,1611604178


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=api_key)

# m = gmaps.Map() #tried to see if maps was working with a basic command, and it wasn't so i used these resources to enable the gmaps extension: https://github.com/pbugnion/gmaps/issues/246; https://github.com/pbugnion/gmaps/issues/79
# m  #WHY ISNT THIS WORKING; basic configuration 

# Store latitude and longitude in locations
locations = weather_data[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = weather_data["Humidity"]  #recall to pull a specific column from the dataframe we bracket the column name around "" since it is a string

In [4]:
# Plot Heatmap
fig = gmaps.figure(center=(40, -20), zoom_level=2) #using gmaps.figure() https://jupyter-gmaps.readthedocs.io/en/latest/api.html
#if you want to download the displayed figure above it is important to tinker with center and zoom values so your image doesn't render cut off; i orignally set center to 0,0 but it cut off the image, so i played around with values until i was satisfied with the end result

max_intensity = np.max(humidity) #for max intensity in the heat map, try setting it to the highest humidity found in the data set.

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=max_intensity, point_radius=3) #https://jupyter-gmaps.readthedocs.io/en/v0.3.3/gmaps.html

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

# Save figure- this was my original thought process but this method is not available since it is a live widget; you can download the picture or take a screenshot
#fig.savefig("../Images/Heatmap w/o Markers.png") 

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equal to 0 and max temp between 65 and 75
ideal_city_df = weather_data.loc[(weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] == 0) & \
                                   (weather_data["Max Temp"] >= 65) & (weather_data["Max Temp"] <= 75)].dropna()

ideal_city_df #display the newly created dataframe 

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
10,Bredasdorp,-34.5322,20.0403,66.20,82,0,4.61,ZA,1611604112
45,Kruisfontein,-34.0033,24.7314,69.01,79,0,1.99,ZA,1611604118
73,Makkah al Mukarramah,21.4267,39.8261,73.71,67,0,6.51,SA,1611603994
138,Hermanus,-34.4187,19.2345,69.01,72,0,7.58,ZA,1611604137
147,Moindou,-21.6924,165.6770,73.40,78,0,4.61,NC,1611604138
207,Saldanha,-33.0117,17.9442,66.20,72,0,9.22,ZA,1611604150
241,Salalah,17.0151,54.0924,71.60,60,0,3.44,OM,1611604156
377,Kidal,18.4411,1.4078,68.04,17,0,9.62,ML,1611604184
421,Hirara,24.8000,125.2833,68.14,56,0,4.72,JP,1611604194
447,Arlit,18.7369,7.3853,66.07,19,0,7.18,NE,1611604199


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create a the hotel dataframe 
hotel_df = ideal_city_df.loc[:,["City","Country", "Lat", "Lng"]] #.loc[:,] all rows with the columns of city,country, lat, and lng

# Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ""  #= "" is how to create a new a columnm name as a placeholder; set a empty string value 

# Display the dataframe
hotel_df

,City,Country,Lat,Lng,Hotel Name
10,Bredasdorp,ZA,-34.5322,20.0403,
45,Kruisfontein,ZA,-34.0033,24.7314,
73,Makkah al Mukarramah,SA,21.4267,39.8261,
138,Hermanus,ZA,-34.4187,19.2345,
147,Moindou,NC,-21.6924,165.6770,
207,Saldanha,ZA,-33.0117,17.9442,
241,Salalah,OM,17.0151,54.0924,
377,Kidal,ML,18.4411,1.4078,
421,Hirara,JP,24.8000,125.2833,
447,Arlit,NE,18.7369,7.3853,


In [7]:
# Make request from google's places api

# Set URL
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json" #this pulled from: https://developers.google.com/places/web-service/search

#Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.

# Set the parameters as required by API

parameters = {"type" : "hotel", # I am searching for hotels therefore i will set the keyword to hotel 
          "keyword" : "hotel",
          "radius" : 5000, # radius will be in meters, according to documentation; asked to search for 5000
          "key" : api_key} # once again, this is the variable name stored in config file

In [8]:
# Make multiple requests from Google's API using a for loop

for index, row in hotel_df.iterrows():  # use iterrows to iterate through pandas dataframe
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to parameters dictionary 
    parameters["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=parameters).json()  #requests.get.json() to get our reponses from API
    
    # extract results
    results = response["results"]
    
    # save the hotel name to the dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.") #[0] is the 0th index of the results dictionary and pull the name of that index
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field can use the except block to accomplish this 
    except (KeyError, IndexError):
        print("Cannot find... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error; researched this as to not exhaust server requests
    time.sleep(1) #https://www.pythoncentral.io/pythons-time-sleep-pause-wait-sleep-stop-your-code/

# Print end of search once search is complete
print("-------Search complete-------")

Retrieving Results for Index 10: Bredasdorp.
Closest hotel in Bredasdorp is Bredasdorp Country Manor.
------------
Retrieving Results for Index 45: Kruisfontein.
Cannot find... skipping.
------------
Retrieving Results for Index 73: Makkah al Mukarramah.
Closest hotel in Makkah al Mukarramah is Pullman Zamzam Makkah.
------------
Retrieving Results for Index 138: Hermanus.
Closest hotel in Hermanus is Ocean Eleven.
------------
Retrieving Results for Index 147: Moindou.
Cannot find... skipping.
------------
Retrieving Results for Index 207: Saldanha.
Closest hotel in Saldanha is Hoedjiesbaai Hotel.
------------
Retrieving Results for Index 241: Salalah.
Closest hotel in Salalah is HYATT SALALAH HOTEL.
------------
Retrieving Results for Index 377: Kidal.
Cannot find... skipping.
------------
Retrieving Results for Index 421: Hirara.
Closest hotel in Hirara is Hotel 385.
------------
Retrieving Results for Index 447: Arlit.
Closest hotel in Arlit is Hôtel Telwa Bungalow.
------------
Re

In [9]:
hotel_df #display the new dataframe with the newly found hotel names 

,City,Country,Lat,Lng,Hotel Name
10,Bredasdorp,ZA,-34.5322,20.0403,Bredasdorp Country Manor
45,Kruisfontein,ZA,-34.0033,24.7314,
73,Makkah al Mukarramah,SA,21.4267,39.8261,Pullman Zamzam Makkah
138,Hermanus,ZA,-34.4187,19.2345,Ocean Eleven
147,Moindou,NC,-21.6924,165.6770,
207,Saldanha,ZA,-33.0117,17.9442,Hoedjiesbaai Hotel
241,Salalah,OM,17.0151,54.0924,HYATT SALALAH HOTEL
377,Kidal,ML,18.4411,1.4078,
421,Hirara,JP,24.8000,125.2833,Hotel 385
447,Arlit,NE,18.7369,7.3853,Hôtel Telwa Bungalow


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer on top of heat map; Plot the hotels on top of the humidity heatmap with each pin containing the Hotel Name, City, and Country.
markers = gmaps.marker_layer(locations, info_box_content = hotel_info) #customizing the info_box field with template created in previous cell

# Add the layer to the map
fig.add_layer(markers)  #add_layer method

# Display Map using fig hanging object 
fig

Figure(layout=FigureLayout(height='420px'))